# Группировки как фильтр
Считаем метрики для пользователей, которые выставили 100 и более оценок

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Упражнение
Получите список пользователей film_fans_user_ids, которые выставили 100 и более оценок фильмам:
1. Для каждого userId посчитайте количество выставленных оценок. Результат сохраните в датафрейм
 ratings_grouped
2. Отфильтруйте пользователей в ratings_grouped, которые поставили 100 и более оценок.
 Результат сохраните в датафрейм ratings_filtered
3. Сохраните идентификаторы пользователей датафрейма ratings_filtered в список film_fans_user_ids:
```python
film_fans_user_ids = ratings_filtered['userId'].tolist()
```
4. Отфильтруйте исходный датафрейм ratings для пользователей из списка film_fans_user_ids:
```python
ratings[ ratings['userId'].isin(...) ]
```

In [2]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings_grouped = ratings.groupby('userId').count()
ratings_filtered = ratings_grouped[ratings_grouped['rating'] >= 100]
# ratings_filtered.sort_values(by=['rating'], ascending=True).head()
film_fans_user_ids = ratings_filtered.reset_index()['userId'].tolist()
more_than_100 = ratings[ ratings['userId'].isin(film_fans_user_ids)]
more_than_100.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Сводные таблицы
Прям как в экселе

In [49]:
data = pd.read_csv('ml-latest-small/ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [50]:
# какие оценки поставил пользователь №1

data[ data['userId'] == 1 ]['rating'].value_counts()

5.0    124
4.0     76
3.0     26
2.0      5
1.0      1
Name: rating, dtype: int64

In [51]:
# а для всех пользователей сразу?

data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,1,0,5,0,26,0,76,0,124
2,0,0,0,1,1,4,4,9,4,6
3,20,0,0,1,0,1,1,1,5,10
4,0,23,0,26,0,39,0,64,0,64
5,0,1,0,3,0,17,0,13,0,10
6,0,7,0,13,0,152,0,102,0,40
7,6,12,12,11,7,16,18,28,29,13
8,0,1,0,3,0,21,0,12,0,10
9,0,6,0,6,0,12,0,14,0,8


In [53]:
# можно итоги добавить
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0,
                 margins = True)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,1,0,5,0,26,0,76,0,124,232
2,0,0,0,1,1,4,4,9,4,6,29
3,20,0,0,1,0,1,1,1,5,10,39
4,0,23,0,26,0,39,0,64,0,64,216
5,0,1,0,3,0,17,0,13,0,10,44
6,0,7,0,13,0,152,0,102,0,40,314
7,6,12,12,11,7,16,18,28,29,13,152
8,0,1,0,3,0,21,0,12,0,10,47
9,0,6,0,6,0,12,0,14,0,8,46


### Упражнение
Какой пользователь выставил больше всех пятерок?

# Объединение датафреймов

In [8]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [9]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [10]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [11]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [12]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [13]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [14]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [15]:
visits_grouped.join(purchases_pivot, 
                    how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [16]:
visits_grouped.join(purchases_pivot, 
                    how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают расходы.

In [17]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [18]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [19]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [20]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [21]:
merged = clients.merge(earnings, how='left').merge(spending, how='left')
govnyuks = merged[merged['credit'] > merged['debit']]['name'].to_list()
print(govnyuks)


['Марков Илья', 'Гордецкий Максим']


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [22]:
visits_grouped.join(purchases_pivot, 
                    how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [23]:
visits_grouped.join(purchases_pivot, 
                    how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


### Дубликаты при объединении таблиц

In [24]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [25]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [26]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [27]:
movies.drop_duplicates(subset = ['movieId'], keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [28]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [29]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Оптимизация хранения данных

In [30]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [31]:
len(ratings) == len(joined)

True

In [32]:
joined.to_csv('joined_ratings.csv', index=False)

### Какой жанр имеет самые высокие рейтинги?

In [33]:
import pandas as pd

In [34]:
genres = ['Drama', 'Action', 'Thriller']

In [35]:
import numpy as np

In [36]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
len(ratings)

100836

In [38]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [39]:
len(movies)

9742

In [40]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [41]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [42]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [43]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [44]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,NaN
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,NaN,4.0,4.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,NaN,NaN,5.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,NaN,NaN,5.0


### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [15]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [16]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [17]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [18]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base


,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


Задание 1
------------------------------------
Посчитайте среднее время жизни пользователей, которые выставили более
 100 оценок. Под временем жизни понимается разница между максимальным и
 минимальным значением столбца timestamp для данного значения userId



In [14]:
timestamp__max = more_than_100.groupby('userId')['timestamp'].max()
timestamp__min = more_than_100.groupby('userId')['timestamp'].min()
lifetime = pd.DataFrame({'min': timestamp__min, 'max': timestamp__max})
lifetime['average'] = lifetime['max'] - lifetime['min']
lifetime['average'].mean()

44190282.03225806

Задание 2
---------
Дана статистика услуг перевозок клиентов компании по
 типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
1. аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [34]:
all_perevozkis = pd.concat([rzd.set_index('client_id'), auto.set_index('client_id'), air.set_index('client_id')],
                           ignore_index=False, sort=False)
all_perevozkis.fillna(0, inplace=True)
all_perevozkis.head()
all_adresses = all_perevozkis.join(client_base.set_index('client_id'), how='left') 
all_adresses.head()
#проверяем
all_perevozkis['rzd_revenue'].sum() == all_adresses['rzd_revenue'].sum()
all_perevozkis['auto_revenue'].sum() == all_adresses['auto_revenue'].sum()

True

Задание 3
------------
В задаче сквозной аналитики вам предоставили данные по местоположению
 пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота),
  когда они требовались приложению для полноценной работы. 
Как бы вы добавили эти сведения в таблицу визитов и покупок?

Решение
-----------
В моем понимании это задача кластеризации.
Таким образом, я выделяю сначала кластеры для каждого пользователя , 
в рамках кластера координаты можно усреднить, а потом привязываю усредненные координаты кластера к визиту 
и покупке через время. Вводим метрику и выбираем ближайшие по времени к покупке или визиту
 координаты класетра, в котором находился пользователь.
В течение дня пользователь оказывается привязанным к одному из кластеров.
Если для пользователя кластера не выявлено - то берем тупо ближайшие координаты, которые удалось снять
и привязываем их.